# 实验：将 MNIST 数据集保存为图片
----

**实验目的：**更深入地理解 `tf.data.Datasets` 数据结构，方便后面进行数据操作。

**实验内容：**将 MNIST 数据集中的前 20 张图片保存到 `./mnist_data/raw/` 下。

## 研究 `tensorflow_datasets` 和 `tf.data.Dataset` 的基本操作

In [86]:
import tensorflow as tf
import tensorflow_datasets as tfds

mnist = tfds.load(name="mnist", data_dir="./mnist_data/")
print(mnist)

{'test': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>, 'train': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>}


> **MNIST 数据集包括：`train` 和 `test` 两个部分，用 `dict` 的数据结构进行组织。可以通过字典操作来分别访问这两个子集。**

In [87]:
mnist_test = mnist["test"]
mnist_train = mnist["train"]

> **或者，在 `load` 时，分别读取这两个子集**

In [88]:
mnist_test = tfds.load(name="mnist", split="test", data_dir="./mnist_data/")
mnist_train = tfds.load(name="mnist", split="train", data_dir="./mnist_data/")

> **`mnist_test` 和 `mnist_train` 都是 `tf.data.Dataset` 类的实例。**
>
> **注意：是 `tf.data.Dataset` 的实例，而不是 `list<tf.data.Dataset>`。**

In [89]:
assert isinstance(mnist_test, tf.data.Dataset)
assert isinstance(mnist_train, tf.data.Dataset)

> - **`tf.data.Dataset` 用于管理数据集的所有数据。**
>
> - **`tf.data.Dataset` 是可迭代的，每次迭代的结果是数据集中的一个数据。**
> 
> - **`tf.data.Dataset` 不支持“下标（subscriptable）”操作，例如：`mnist_test[0]` 是不允许的。**
>
> - **可以使用 `for` 来进行遍历 `tf.data.Dataset` 中的数据。**
>
> - **可以使用 `take()` 类方法，来生成 `tf.data.Dataset` 实例的一个子集（仍然是一个 `tf.data.Dataset` 实例）**
>
> - **帮助生成子集的类方法，还包括：`skip()` 和 `range()`**

In [90]:
# 获取 mnist_train 中的第一个数据的数据结构
for el in mnist_train.take(1):
    print(el.keys())

dict_keys(['image', 'label'])


2022-01-20 20:49:44.371885: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


> **数据集中的数据是一个 `dict`。在 MNIST 数据集中，每个数据包括两个字段：`image` 和 `label`。**

In [91]:
# 获取 mnist_test 中前10个数据的 label
for el in mnist_test.take(10):
    print(el["label"])

tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)


2022-01-20 20:49:44.411593: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [92]:
# 获取 mnist_test 中第三个数据的 label
for el in mnist_test.skip(2).take(1):
    print(el["label"])

tf.Tensor(4, shape=(), dtype=int64)


2022-01-20 20:49:44.453961: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


> **可以将 `tf.data.Datasets` 转换成 `list` 来访问数据。**
>
> **注意：对于大数据集转换成 `list` 非常耗时，不如直接对 `tf.data.Dataset` 进行操作。**

In [93]:
# 获取 mnist_test 中前10个数据的 label
for el in list(mnist_test)[:10]:
    print(el["label"])

tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)


## 保存 `mnist_train` 中的前 20 张图片 

In [95]:
import tensorflow as tf
import tensorflow_datasets as tfds
import imageio
import os

# 设置科学上网
os.environ["http_proxy"] = "http://127.0.0.1:1081"
os.environ["https_proxy"] = "http://127.0.0.1:1081"

# 读取 MNIST train 数据集，如果不存在会事先下载
mnist_train = tfds.load(name="mnist", split="train", data_dir="./mnist_data/")

# 把原始图片保存在 mnist_data/raw/ 目录下
# 如果没有这个文件夹，则创建它
save_dir = "mnist_data/raw/"
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)
    
# 保存前 20 张图片
i = 0
for el in mnist_train.take(20):
    # tensorflow 中处理的都是张量（Tensor）。为了保存图片，需将其转换成 array
    # MNIST 中图片的 shape = (28, 28, 1)，表示是一张 28x28 的灰度图片
    image_array = el["image"].numpy()
    
    # 图像文件的文件名格式为：
    # mnist_train_0.jpg, mnist_train_1.jpg, ..., mnist_train_19.jpg
    filename = save_dir + "mnist_train_%d.jpg" % i
    
    # 将 image_array 保存为图片
    imageio.imsave(filename, image_array)
    
    # 更新文件索引
    i += 1
    

2022-01-20 20:50:07.163311: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
